In [1]:
import pandas as pd
import numpy as np
import os
import requests
from time import sleep
from collections import Counter
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from config import db_type, db_user, db_pass, db_project, db_aws

db_address = db_type + db_user + db_pass + db_project + db_aws

In [371]:
reviews= pd.read_csv("google_vineyard_reviews.csv")
details=pd.read_csv("google_vineyard_details.csv")
photos=pd.read_csv("google_vineyard_photos.csv")
geometry=pd.read_csv("google_vineyard_maps_geometry.csv")

In [372]:
details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   business_status         659 non-null    object 
 1   formatted_phone_number  606 non-null    object 
 2   name                    661 non-null    object 
 3   place_id                661 non-null    object 
 4   rating                  581 non-null    float64
 5   user_ratings_total      581 non-null    float64
 6   formatted_address       661 non-null    object 
 7   website                 585 non-null    object 
 8   street_address_1        589 non-null    object 
 9   street_address_2        0 non-null      float64
 10  city                    589 non-null    object 
 11  province                589 non-null    object 
 12  zipcode                 589 non-null    float64
 13  country_code            589 non-null    object 
 14  url                     661 non-null    ob

In [373]:
name = details.loc[[i for i in details.index if len(details["name"][i]) >30],]["name"]
name

16     Armstrong Family Winery - Downtown Walla Walla...
17     Los Rocosos Vineyards-The Rocks District of Mi...
35               Eagles Nest Reserve Vineyard and Winery
56               Chehalem Winery Tasting Room & Wine Bar
61                Archer Vineyard, Winery & Tasting Room
                             ...                        
607        Springhill Cellars Winery (Open May-December)
621                 Territorial Vineyards & Wine Company
645    Rosella's Vineyard & Winery And Home Of SPRETZELS
651                Evening Land - Seven Springs Vineyard
652                     Meadows Estate Vineyard & Winery
Name: name, Length: 65, dtype: object

In [374]:
name.index

Int64Index([ 16,  17,  35,  56,  61,  76,  88,  91,  94, 100, 106, 112, 121,
            122, 128, 146, 158, 203, 219, 235, 241, 252, 269, 273, 281, 287,
            294, 335, 339, 349, 353, 371, 383, 399, 401, 409, 410, 430, 443,
            450, 462, 463, 470, 477, 493, 499, 507, 509, 512, 514, 527, 554,
            556, 557, 570, 571, 580, 583, 600, 604, 607, 621, 645, 651, 652],
           dtype='int64')

In [375]:
## variable for the location name to distingush multiple locations
location_name=name[[16,17,76,94,128,203,241,273,349,353,401,463,527,554,570,571,580,651]]
location_name[16] = location_name[16][26:]
location_name[17]= location_name[17][22:]
location_name[76]= location_name[76][13:]
location_name[94]= location_name[94][15:]
location_name[128]= location_name[128][5:]
location_name[203]= location_name[203][40:]
location_name[241]= location_name[241][31:]
location_name[273]= location_name[273]
location_name[349]= location_name[349][30:49]
location_name[353]= location_name[353][27:]
location_name[401]= location_name[401][22:]
location_name[463]= location_name[463][22:]
location_name[527]= location_name[527][22:-1]
location_name[554]= location_name[554][28:]
location_name[570]= location_name[570][14:]
location_name[571]= location_name[571][22:-1]
location_name[580]= location_name[580][28:]
location_name[651]= location_name[651][15:]
location_name

16           Downtown Walla Walla Tasting Room
17      The Rocks District of Milton Freewater
76           Signature Tasting on the Vineyard
94                     Wine Lounge in Portland
128          Tasting Room & Wine Bar Hillsboro
203                   Woodinville Tasting Room
241                               Tualatin, OR
273    Portland Wine Bar & Winery Tasting Room
349                        Carlton Wine Corner
353                        Dundee Tasting Room
401                            Woodinville, WA
463                       Sams Valley Vineyard
527                      Downtown Tasting Room
554                   McMinnville Tasting Room
570                   Tasting Room Walla Walla
571                     Winery at the Vineyard
580      Lake Oswego Tasting Room & Restaurant
651                     Seven Springs Vineyard
Name: name, dtype: object

In [376]:
name[16] = name[16][:23]
name[17] = name[17][:21]
name[76] = name[76][:12]
name[94] = name[94][:14]
name[122] = name[122][:15]
name[128] = name[128][:4]
name[203] = name[203][:39]
name[241] = name[241][:28]
name[273] = "The Winery at Manzanita"
name[349] = name[349][:27]
name[353] = name[353][:24]
name[401] = name[401][:20]
name[410] = name[410][:17]
name[463] = name[463][:19]
name[499] = name[499][:23]
name[527] = name[527][:20]
name[554] = name[554][:27]
name[570] = name[570][:13]
name[571] = name[571][:20]
name[580] = name[580][:27]
name[607] = name[607][:25]
name[645] = name[645][:-22]
name[651] = name[651][:12]
name

16                     Armstrong Family Winery
17                       Los Rocosos Vineyards
35     Eagles Nest Reserve Vineyard and Winery
56     Chehalem Winery Tasting Room & Wine Bar
61      Archer Vineyard, Winery & Tasting Room
                        ...                   
607                  Springhill Cellars Winery
621       Territorial Vineyards & Wine Company
645                Rosella's Vineyard & Winery
651                               Evening Land
652           Meadows Estate Vineyard & Winery
Name: name, Length: 65, dtype: object

In [377]:
for i in name.index:
    details['name'][i] = name[i]

C:\Users\Nick\AppData\Local\Temp\ipykernel_304\2355944396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details['name'][i] = name[i]


In [378]:
details = details.replace({np.nan:"NULL"})
for i in location_name.index:
    details['location_name'][i] = location_name[i]

In [379]:
test = "'"
for i in details.index:
    if test in details['name'][i]:
        print(i, "   ||   ", details['name'][i])



21    ||    Miller's Vineyard
28    ||    Wy'East Vineyards
106    ||    Dragon's Vineyard & Wine Company
110    ||    St Josef's Winery
147    ||    L'Angolo Estate
158    ||    Pete's Mountain Vineyard and Winery
281    ||    Whiskey Hill Winery & Postlewait's Vineyards
315    ||    Sophie's Vineyard
334    ||    Paul O'Brien Winery
342    ||    Idiot's Grace Wines
447    ||    Natalie's Estate Winery
509    ||    Teso'aria Vineyard and Winery Annex
520    ||    River's Edge Winery
561    ||    Cana's Feast Winery
622    ||    Winter's Hill Estate
626    ||    King's Raven Winery
645    ||    Rosella's Vineyard & Winery


In [380]:
name = details.loc[[i for i in details.index if test in details['name'][i]], 'name']
for i in name.index:
    name[i] =name[i].replace(test, "")

name    

21                                Millers Vineyard
28                                WyEast Vineyards
106                Dragons Vineyard & Wine Company
110                               St Josefs Winery
147                                 LAngolo Estate
158             Petes Mountain Vineyard and Winery
281    Whiskey Hill Winery & Postlewaits Vineyards
315                               Sophies Vineyard
334                             Paul OBrien Winery
342                             Idiots Grace Wines
447                         Natalies Estate Winery
509             Tesoaria Vineyard and Winery Annex
520                             Rivers Edge Winery
561                             Canas Feast Winery
622                            Winters Hill Estate
626                             Kings Raven Winery
645                     Rosellas Vineyard & Winery
Name: name, dtype: object

In [381]:
for i in name.index:
    details['name'][i] = name[i]

In [382]:
for i, x in enumerate(details["formatted_address"]):
    broken = x.split(",")
    details['country_code'][i] = broken[-1]
    broken2 = broken[-2].split()
    details['province'][i] = broken2[0]
    details['zipcode'][i] = broken2[1]
    details['city'][i] = broken[-3]
    
    if len(broken[:-3]) < 2:
        if len(broken[:-3]) !=0: 
            details['street_address_1'][i] = broken[:-3][0]
        else:
            details['street_address_1'][i] = "NULL"
    elif i in [153,214, 275,447,508,657]:
        details['street_address_1'][i] = broken[:-3][-1]
    else:
        details['street_address_1'][i] = "".join(broken[:-3])
    
    

In [385]:
details['business_status'].unique()

array(['OPERATIONAL', 'CLOSED_PERMANENTLY', 'NULL', 'CLOSED_TEMPORARILY'],
      dtype=object)

In [386]:
details = details[details["business_status"]!="CLOSED_PERMANENTLY"]


In [387]:
details.to_csv("google_vineyard_details.csv", index=False)

In [388]:
details = details[details['province']=='OR']
details

,business_status,formatted_phone_number,name,place_id,rating,user_ratings_total,formatted_address,website,street_address_1,street_address_2,city,province,zipcode,country_code,url,location_name
2,OPERATIONAL,(503) 363-0525,Cória Estates,ChIJm-BEGj4GwFQRQz0MEeIFSLM,4.7,80.0,"8252 Redstone Ave SE, Salem, OR 97306, USA",http://www.coriaestates.com/,8252 Redstone Ave SE,NULL,Salem,OR,97306,USA,https://maps.google.com/?cid=12918581999619030339,NULL
3,OPERATIONAL,(541) 928-2106,Vivacity Spirits LLC,ChIJ6QmAe3sVwFQR7KVJoo1dw3k,4.9,13.0,"140 NE Hill St, Albany, OR 97321, USA",http://www.vivacityspirits.com/,140 NE Hill St,NULL,Albany,OR,97321,USA,https://maps.google.com/?cid=8773959361942365676,NULL
4,OPERATIONAL,(541) 929-3519,Lumos Wine Co.,ChIJQeW2X8JIlVQR_36_D7DaRyw,NULL,NULL,"845 NE 5th St, McMinnville, OR 97128, USA",http://www.lumoswine.com/,845 NE 5th St,NULL,McMinnville,OR,97128,USA,https://maps.google.com/?cid=3190759310728593151,NULL
5,OPERATIONAL,(503) 679-3497,Gresser Vineyard,ChIJPyUzT_0WlVQRPplhbRhoBv8,5.0,3.0,"37245 SW Nature Dr, Cornelius, OR 97113, USA",NULL,37245 SW Nature Dr,NULL,Cornelius,OR,97113,USA,https://maps.google.com/?cid=18376489783655504190,NULL
7,OPERATIONAL,(503) 852-3185,Pike Road Wines,ChIJsQUl8WI4lVQRH7c1h2mlyxs,4.8,76.0,"701 NE 3rd St, McMinnville, OR 97128, USA",https://www.pikeroadwines.com/About,701 NE 3rd St,NULL,McMinnville,OR,97128,USA,https://maps.google.com/?cid=2002876331954911007,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,OPERATIONAL,(503) 203-2583,Seven Bridges Winery,ChIJt-n5RFqnlVQR5DwhGTve1hk,4.6,34.0,"2303 N Harding Ave, Portland, OR 97227, USA",http://www.sevenbridgeswinery.com/,2303 N Harding Ave,NULL,Portland,OR,97227,USA,https://maps.google.com/?cid=1861919841370324196,NULL
657,OPERATIONAL,(503) 537-1137,Beaux Frères,ChIJY_YvLqs_lVQRQ1JGKPCC-qA,4.9,57.0,"By Appointment Only, 15155 NE North Valley Rd,...",http://www.beauxfreres.com/,15155 NE North Valley Rd,NULL,Newberg,OR,97132,USA,https://maps.google.com/?cid=11599727758225592899,NULL
658,OPERATIONAL,(541) 942-8028,Chateau Lorane,ChIJNdln3D05wVQRP5h-7LTPjaI,4.8,25.0,"27415 Siuslaw Rd, Lorane, OR 97451, USA",https://www.chateaulorane.com/,27415 Siuslaw Rd,NULL,Lorane,OR,97451,USA,https://maps.google.com/?cid=11713246581829310527,NULL
659,OPERATIONAL,(800) 245-2950,The Four Graces Winery,ChIJsfC-ZK9BlVQRdC0UjH76x58,4.6,94.0,"9605 NE Fox Farm Rd, Dundee, OR 97115, USA",https://www.thefourgraces.com/,9605 NE Fox Farm Rd,NULL,Dundee,OR,97115,USA,https://maps.google.com/?cid=11513446393818918260,NULL


In [424]:
reviews

,author_name,author_url,original_language,rating,relative_time_description,text,time,translated,place_id
0,Smiledoc,https://www.google.com/maps/contrib/1063067683...,en,5,4 years ago,"great wine, great people, great location and f...",1522099905,False,ChIJb21eb2cVolQRY6JGmq_XmwA
1,Kay Ross,https://www.google.com/maps/contrib/1123279401...,en,5,a year ago,First time at Coria and not disappointed!\nCai...,1628467472,False,ChIJm-BEGj4GwFQRQz0MEeIFSLM
2,Linda Teppler (S0rceress0),https://www.google.com/maps/contrib/1049795854...,en,5,3 years ago,Just tried their Treos Brandy at the Corvallis...,1561834276,False,ChIJ6QmAe3sVwFQR7KVJoo1dw3k
3,Matias Sanchez,https://www.google.com/maps/contrib/1034946324...,en,5,4 years ago,The wine here blows my mind away!,1533678636,False,ChIJPyUzT_0WlVQRPplhbRhoBv8
4,Michael Hunter,https://www.google.com/maps/contrib/1134729068...,en,5,3 months ago,New tasting room on 3rd Street in mcminnville....,1659104687,False,ChIJsQUl8WI4lVQRH7c1h2mlyxs
...,...,...,...,...,...,...,...,...,...
567,Cameron Winchester,https://www.google.com/maps/contrib/1036651225...,en,5,6 months ago,Really fantastic wines in the city! Staff is w...,1650850207,False,ChIJt-n5RFqnlVQR5DwhGTve1hk
568,Kate S.,https://www.google.com/maps/contrib/1004652575...,en,4,5 months ago,We must have had an off day when we visited. T...,1653419759,False,ChIJY_YvLqs_lVQRQ1JGKPCC-qA
569,Danielle Verdugo,https://www.google.com/maps/contrib/1013689582...,en,5,8 years ago,"A must see and taste!! Wonderful staff, beauti...",1407261261,False,ChIJNdln3D05wVQRP5h-7LTPjaI
570,Alaina Danae,https://www.google.com/maps/contrib/1045108330...,en,5,3 years ago,This winery is absolutely amazing! The staff i...,1543869171,False,ChIJsfC-ZK9BlVQRdC0UjH76x58


In [430]:
reviews = reviews.replace(np.nan, 'NULL')

for i, text in enumerate(reviews['text']):
    try: 
        if test in text:
            reviews['text'][i] = text.replace(test, "")
    except TypeError: 
        print("============\n", f"TypeERROR:   {i} and text =>   \n{text}\n", "===============")


C:\Users\Nick\AppData\Local\Temp\ipykernel_304\45381148.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['text'][i] = text.replace(test, "")


In [434]:
reviews.to_csv("google_vineyard_reviews.csv")

In [326]:
engine = create_engine(db_address)
inspector = inspect(engine)
Base = automap_base()
Base.prepare(engine, reflect=True)

In [351]:
conn = engine.connect()

In [389]:
# place_id, business_name,location_name, phone, rating, user_ratings_total, formatted_address, street_address,
#  city, province, zipcode, country, region, subregion, status, gmaps_url

for i in details.index[:21]:
        conn.execute(f"""INSERT INTO vineyards (place_id, business_name, phone, rating, user_ratings_total, status, gmaps_url, formatted_address, 
                                            street_address, city, province, zipcode, country_code, region, subregion)
                     VALUES ('{details["place_id"][i]}', '{details['name'][i]}', '{details["formatted_phone_number"][i]}', {details["rating"][i]}, 
                              {details['user_ratings_total'][i]}, '{details['business_status'][i]}', '{details['url'][i]}', '{details['formatted_address'][i]}', 
                             '{details["street_address_1"][i]}', '{details["city"][i]}', '{details['province'][i]}', '{details["zipcode"][i]}', 
                             '{details['country_code'][i]}', NULL, NULL);
                    """)
    sleep(1)



IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "vineyards_pkey"
DETAIL:  Key (place_id)=(ChIJm-BEGj4GwFQRQz0MEeIFSLM) already exists.

[SQL: INSERT INTO vineyards (place_id, business_name, phone, rating, user_ratings_total, status, gmaps_url, formatted_address, 
                                            street_address, city, province, zipcode, country_code, region, subregion)
                     VALUES ('ChIJm-BEGj4GwFQRQz0MEeIFSLM', 'Cória Estates', '(503) 363-0525', 4.7, 
                              80.0, 'OPERATIONAL', 'https://maps.google.com/?cid=12918581999619030339', '8252 Redstone Ave SE, Salem, OR 97306, USA', 
                             '8252 Redstone Ave SE', ' Salem', 'OR', '97306', 
                             ' USA', NULL, NULL);

                    ]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [365]:
query = conn.execute("Select * from vineyards;")
for i, r in enumerate(query):
    print("==========\n", i, "\n", r)

 0 
 ('ChIJb21eb2cVolQRY6JGmq_XmwA', 'K Vintners', None, '(509) 526-5230', 4.4, 13, '820 Mill Creek Rd, Walla Walla, WA 99362, USA', '820 Mill Creek Rd', ' Walla Walla', 'WA', '99362', ' USA', None, None, 'OPERATIONAL', 'https://maps.google.com/?cid=43865770597720675')
 1 
 ('ChIJZdNgbQb3xVQRba7OWrKWhno', 'LaBrasseur Vineyard', None, '(541) 865-3648', None, None, '2444 Cobleigh Rd, Eagle Point, OR 97524, USA', '2444 Cobleigh Rd', ' Eagle Point', 'OR', '97524', ' USA', None, None, 'CLOSED_PERMANENTLY', 'https://maps.google.com/?cid=8828909812278275693')
 2 
 ('ChIJm-BEGj4GwFQRQz0MEeIFSLM', 'Cória Estates', None, '(503) 363-0525', 4.7, 80, '8252 Redstone Ave SE, Salem, OR 97306, USA', '8252 Redstone Ave SE', ' Salem', 'OR', '97306', ' USA', None, None, 'OPERATIONAL', 'https://maps.google.com/?cid=12918581999619030339')
 3 
 ('ChIJ6QmAe3sVwFQR7KVJoo1dw3k', 'Vivacity Spirits LLC', None, '(541) 928-2106', 4.9, 13, '140 NE Hill St, Albany, OR 97321, USA', '140 NE Hill St', ' Albany', 'OR', '

In [395]:
# place_id, business_name,location_name, phone, rating, user_ratings_total, formatted_address, street_address,
#  city, province, zipcode, country, region, subregion, status, gmaps_url

for i in details.index[21:]:
    conn.execute(f"""INSERT INTO vineyards (place_id, business_name, phone, rating, user_ratings_total, status, gmaps_url, formatted_address, 
                                            street_address, city, province, zipcode, country_code, region, subregion)
                     VALUES ('{details["place_id"][i]}', '{details['name'][i]}', '{details["formatted_phone_number"][i]}', {details["rating"][i]}, 
                              {details['user_ratings_total'][i]}, '{details['business_status'][i]}', '{details['url'][i]}', '{details['formatted_address'][i]}', 
                             '{details["street_address_1"][i]}', '{details["city"][i]}', '{details['province'][i]}', '{details["zipcode"][i]}', 
                             '{details['country_code'][i]}', NULL, NULL);
                    """)
    sleep(1)



IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "vineyards_pkey"
DETAIL:  Key (place_id)=(ChIJycscKsxGlVQRImdkggvMr0Y) already exists.

[SQL: INSERT INTO vineyards (place_id, business_name, phone, rating, user_ratings_total, status, gmaps_url, formatted_address, 
                                            street_address, city, province, zipcode, country_code, region, subregion)
                     VALUES ('ChIJycscKsxGlVQRImdkggvMr0Y', 'White Rose Estate', '(503) 864-2328', 4.7, 
                              144.0, 'OPERATIONAL', 'https://maps.google.com/?cid=5093514053383644962', '6250 NE Hilltop Ln, Dayton, OR 97114, USA', 
                             '6250 NE Hilltop Ln', ' Dayton', 'OR', '97114', 
                             ' USA', NULL, NULL);
                    ]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [370]:
details.loc[175:183,]

,business_status,formatted_phone_number,name,place_id,rating,user_ratings_total,formatted_address,website,street_address_1,street_address_2,city,province,zipcode,country_code,url,location_name
175,OPERATIONAL,(503) 580-1596,Knudsen Vineyards,ChIJJ1AqhglBlVQRYYqcPLUg6eE,4.9,83.0,"9419 NE Worden Hill Rd, Dundee, OR 97115, USA",https://knudsenvineyards.com/?utm_source=GMB%2...,9419 NE Worden Hill Rd,NULL,Dundee,OR,97115,USA,https://maps.google.com/?cid=16278578290885429857,NULL
177,OPERATIONAL,(503) 610-2881,Gemini Vineyards,ChIJHbrgbEYTlVQRlDMFCju7t2w,NULL,NULL,"21133 SW Lebeau Rd, Sherwood, OR 97140, USA",http://www.geminivineyard.com/,21133 SW Lebeau Rd,NULL,Sherwood,OR,97140,USA,https://maps.google.com/?cid=7833935939079844756,NULL
179,OPERATIONAL,(503) 714-2030,Archery Summit,ChIJV1ZPvUtBlVQRu6PnF0iWLnc,4.6,94.0,"18599 NE Archery Summit Rd, Dayton, OR 97114, USA",https://www.archerysummit.com/,18599 NE Archery Summit Rd,NULL,Dayton,OR,97114,USA,https://maps.google.com/?cid=8587966775824982971,NULL
180,OPERATIONAL,(503) 873-9800,Vitis Ridge,ChIJ9eRassTjv1QRBM49eT3Jr5Y,5.0,40.0,"6685 Meridian Rd NE, Silverton, OR 97381, USA",http://www.vitisridge.com/,6685 Meridian Rd NE,NULL,Silverton,OR,97381,USA,https://maps.google.com/?cid=10858118492477836804,NULL
181,OPERATIONAL,(503) 434-9766,Advanced Vineyard Systems Inc,ChIJlTazwd1IlVQRwTyolVvBOBw,4.5,4.0,"4651 NE Riverside Dr, McMinnville, OR 97128, USA",http://advancedvineyards.com/,4651 NE Riverside Dr,NULL,McMinnville,OR,97128,USA,https://maps.google.com/?cid=2033587830854794433,NULL
182,OPERATIONAL,NULL,"Blue Heron Vineyards, LLC",ChIJgdjxjCEjxFQR9qCx7V8hYfY,5.0,1.0,"3440 Garden Valley Rd, Roseburg, OR 97471, USA",http://www.bhvin.com/,3440 Garden Valley Rd,NULL,Roseburg,OR,97471,USA,https://maps.google.com/?cid=17753507901964656886,NULL
183,OPERATIONAL,(503) 864-8777,Remy Wines Tasting Room,ChIJY8_oWaRGlVQRD_kczBzbt-8,4.6,83.0,"17495 NE McDougall Rd, Dayton, OR 97114, USA",https://remywines.com/,17495 NE McDougall Rd,NULL,Dayton,OR,97114,USA,https://maps.google.com/?cid=17273515812532844815,NULL


## upload status update
Details dataframe locations are uploading, index 179 has a dataype issue i cannot find. will attempt to look into this after the majority of the data is in the sql database

In [397]:
# place_id, business_name,location_name, phone, rating, user_ratings_total, formatted_address, street_address,
#  city, province, zipcode, country, region, subregion, status, gmaps_url

for i in details.index[180:]:
    conn.execute(f"""INSERT INTO vineyards (place_id, business_name, phone, rating, user_ratings_total, status, gmaps_url, formatted_address, 
                                            street_address, city, province, zipcode, country_code, region, subregion)
                     VALUES ('{details["place_id"][i]}', '{details['name'][i]}', '{details["formatted_phone_number"][i]}', {details["rating"][i]}, 
                              {details['user_ratings_total'][i]}, '{details['business_status'][i]}', '{details['url'][i]}', '{details['formatted_address'][i]}', 
                             '{details["street_address_1"][i]}', '{details["city"][i]}', '{details['province'][i]}', '{details["zipcode"][i]}', 
                             '{details['country_code'][i]}', NULL, NULL);
                    """)
    sleep(1)



In [402]:
geometry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   center_lat     636 non-null    float64
 1   center_lon     636 non-null    float64
 2   northeast_lat  636 non-null    float64
 3   northeast_lon  636 non-null    float64
 4   southwest_lat  636 non-null    float64
 5   southwest_lon  636 non-null    float64
 6   place_id       636 non-null    object 
dtypes: float64(6), object(1)
memory usage: 34.9+ KB


In [435]:
query = conn.execute("SELECT place_id FROM vineyards;")
good_places = {row[0] for row in query}
good_places

{'ChIJ-1_CKzMwwVQRFbP5wXVgQIc',
 'ChIJ-3NYYMJIlVQRVHh3ddXKYZ8',
 'ChIJ-7nIyYkDwFQRjebO6KnKurk',
 'ChIJ-9G5FrpDolQRdju0bfr0ckk',
 'ChIJ-S-78vN4z1QRQt5YVZ9DqBQ',
 'ChIJ-U6JGsKFz1QRARMuqdX1vU0',
 'ChIJ-W8OECriwFQRkNvt5E2iPxY',
 'ChIJ-YWFpPmH6lQR__gssuqycuo',
 'ChIJ-cbq2qUxwVQRrpkiGzmha1s',
 'ChIJ-drpIApBlVQR9Z9q79A9WNM',
 'ChIJ-zAmx1Q_lVQRBsTuiojvb_Q',
 'ChIJ-zfS-Kb3wFQR76Ls6oSY6YQ',
 'ChIJ0837TqdBlVQR8w_X_SPvVx4',
 'ChIJ08Rvf9tJlVQRxurM-mtG6OU',
 'ChIJ09XGBkBAlVQRF5HO7CAhfMg',
 'ChIJ0Wg7T5yq6lQRJKYZZfiLfNY',
 'ChIJ0WqRrXv7v1QRRW4uXuXu8s8',
 'ChIJ0X9UQFoXwFQRinOBT3psfNM',
 'ChIJ0ZownihQlVQRRVUlJPaayio',
 'ChIJ0eUuQHhRlVQRARsCNSh7sGQ',
 'ChIJ181L7RJBlVQRUPxoitr7l_c',
 'ChIJ193bfNCp6lQRSpMFXAXMMG0',
 'ChIJ1Tdr_zLhv1QRTWGWVLyzVC0',
 'ChIJ1UIxLCtBlVQRsGp3EEosvaE',
 'ChIJ1UekEQUKlVQRvCOAz-r0raI',
 'ChIJ1UjYcdOr6lQRxWbO06g1SmE',
 'ChIJ1_Wx2YM4wFQR_XpeBVyu5OY',
 'ChIJ1ei999hyz1QRXmFqtR6TDYw',
 'ChIJ1zXRoBARlVQRpiVR2qIODsw',
 'ChIJ22vW2sjFolQRohtq-192_d0',
 'ChIJ24f4QNc_lVQRYnXv6Aepxo8',
 'ChIJ27

In [408]:
locations = geometry.loc[[i for i in geometry.index if geometry['place_id'][i] in good_places],]
locations

,center_lat,center_lon,northeast_lat,northeast_lon,southwest_lat,southwest_lon,place_id
1,44.833859,-123.047117,44.835195,-123.045148,44.832497,-123.053022,ChIJm-BEGj4GwFQRQz0MEeIFSLM
2,44.639261,-123.094003,44.640652,-123.092491,44.637954,-123.095188,ChIJ6QmAe3sVwFQR7KVJoo1dw3k
3,45.211577,-123.190135,45.212852,-123.188785,45.210154,-123.191483,ChIJQeW2X8JIlVQR_36_D7DaRyw
4,45.444671,-123.061066,45.445973,-123.059668,45.443275,-123.062366,ChIJPyUzT_0WlVQRPplhbRhoBv8
5,45.210161,-123.191821,45.211438,-123.190469,45.208741,-123.193167,ChIJsQUl8WI4lVQRH7c1h2mlyxs
...,...,...,...,...,...,...,...
631,45.539489,-122.678372,45.540765,-122.676951,45.538067,-122.679649,ChIJt-n5RFqnlVQR5DwhGTve1hk
632,45.347624,-123.093657,45.348952,-123.092368,45.346254,-123.095066,ChIJY_YvLqs_lVQRQ1JGKPCC-qA
633,43.847431,-123.250155,43.848787,-123.248608,43.846089,-123.251306,ChIJNdln3D05wVQRP5h-7LTPjaI
634,45.285784,-123.004797,45.287135,-123.003267,45.284437,-123.005965,ChIJsfC-ZK9BlVQRdC0UjH76x58


In [410]:
for i in locations.index:
    conn.execute(f"""INSERT INTO geometry (place_id, center_lat, center_lon, northeast_lat, northeast_lon, southwest_lat, southwest_lon)
         VALUES ('{geometry["place_id"][i]}', {geometry["center_lat"][i]}, {geometry["center_lon"][i]}, {geometry["northeast_lat"][i]},
                 {geometry["northeast_lon"][i]}, {geometry["southwest_lat"][i]}, {geometry["southwest_lon"][i]});
        """)
    sleep(1)
    if i % 23:
        print("Index {i}: complete") ## Forgot the f to make the substitution work

Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete


Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete
Index {i}: complete


In [433]:
place_reviews = reviews.loc[[i for i in reviews.index if reviews['place_id'][i] in good_places],]
place_reviews

,author_name,author_url,original_language,rating,relative_time_description,text,time,translated,place_id
1,Kay Ross,https://www.google.com/maps/contrib/1123279401...,en,5,a year ago,First time at Coria and not disappointed!\nCai...,1628467472,False,ChIJm-BEGj4GwFQRQz0MEeIFSLM
2,Linda Teppler (S0rceress0),https://www.google.com/maps/contrib/1049795854...,en,5,3 years ago,Just tried their Treos Brandy at the Corvallis...,1561834276,False,ChIJ6QmAe3sVwFQR7KVJoo1dw3k
3,Matias Sanchez,https://www.google.com/maps/contrib/1034946324...,en,5,4 years ago,The wine here blows my mind away!,1533678636,False,ChIJPyUzT_0WlVQRPplhbRhoBv8
4,Michael Hunter,https://www.google.com/maps/contrib/1134729068...,en,5,3 months ago,New tasting room on 3rd Street in mcminnville....,1659104687,False,ChIJsQUl8WI4lVQRH7c1h2mlyxs
5,Emily,https://www.google.com/maps/contrib/1161315244...,en,5,a year ago,Great fruit wine and regular wine for holidays...,1605990070,False,ChIJFxlq4W45xFQRofB1FANAgqA
...,...,...,...,...,...,...,...,...,...
567,Cameron Winchester,https://www.google.com/maps/contrib/1036651225...,en,5,6 months ago,Really fantastic wines in the city! Staff is w...,1650850207,False,ChIJt-n5RFqnlVQR5DwhGTve1hk
568,Kate S.,https://www.google.com/maps/contrib/1004652575...,en,4,5 months ago,We must have had an off day when we visited. T...,1653419759,False,ChIJY_YvLqs_lVQRQ1JGKPCC-qA
569,Danielle Verdugo,https://www.google.com/maps/contrib/1013689582...,en,5,8 years ago,"A must see and taste!! Wonderful staff, beauti...",1407261261,False,ChIJNdln3D05wVQRP5h-7LTPjaI
570,Alaina Danae,https://www.google.com/maps/contrib/1045108330...,en,5,3 years ago,This winery is absolutely amazing! The staff i...,1543869171,False,ChIJsfC-ZK9BlVQRdC0UjH76x58


In [445]:
# place_id, author_name, author_url, original_language, translated, utc_time, rating, review_text

for i in place_reviews.index:
    conn.execute(f"""INSERT INTO reviews (place_id, author_name, author_url, original_language, translated, utc_time, rating, review_text)
         VALUES ('{reviews["place_id"][i]}', '{reviews["author_name"][i]}', '{reviews["author_url"][i]}', '{reviews["original_language"][i]}',
                 {reviews["translated"][i]}, {reviews["time"][i]}, {reviews["rating"][i]}, '{reviews["text"][i]}' );
        """)
    sleep(1)
    if i % 14==0:
        print(f"Index {i}: complete") ## Did not Forgot the f to make the substitution work this time.

Index 42: complete
Index 56: complete
Index 70: complete
Index 84: complete
Index 98: complete
Index 112: complete
Index 126: complete
Index 140: complete
Index 154: complete
Index 168: complete
Index 182: complete
Index 196: complete
Index 210: complete
Index 224: complete
Index 238: complete
Index 266: complete
Index 294: complete
Index 308: complete
Index 336: complete
Index 350: complete
Index 364: complete
Index 378: complete
Index 392: complete
Index 420: complete


TypeError: dict is not a sequence

In [448]:
## Type Error at index 428, Same TypeError with Details.index 179.
## Continue from index 429.

# place_id, author_name, author_url, original_language, translated, utc_time, rating, review_text

for i in place_reviews.index[429:]:
    conn.execute(f"""INSERT INTO reviews (place_id, author_name, author_url, original_language, translated, utc_time, rating, review_text)
         VALUES ('{reviews["place_id"][i]}', '{reviews["author_name"][i]}', '{reviews["author_url"][i]}', '{reviews["original_language"][i]}',
                 {reviews["translated"][i]}, {reviews["time"][i]}, {reviews["rating"][i]}, '{reviews["text"][i]}' );
        """)
    sleep(1)
    if i % 14==0:
        print(f"Index {i}: complete") ## Did not Forgot the f to make the substitution work this time.

Index 532: complete


ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "malley"
LINE 2: ...VALUES ('ChIJ9QK5-RlBlVQR0YyVd0Jlv-8', 'shannon o'malley', '...
                                                             ^

[SQL: INSERT INTO reviews (place_id, author_name, author_url, original_language, translated, utc_time, rating, review_text)
         VALUES ('ChIJ9QK5-RlBlVQR0YyVd0Jlv-8', 'shannon o'malley', 'https://www.google.com/maps/contrib/112148964833392878468/reviews', 'en',
                 False, 1662432660, 5, 'The absolute best Pinot I’ve tried in the area. Lyla was a gem and very knowledgeable. She even showed us around and let us try grapes- I highly recommend coming here for the ambiance and family vibe they have!' );
        ]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [475]:
# #  Error at index 549 for an ' in the author_name, fix that and continue, there are no more in the names or text
place = [i for i in place_reviews.index[429:] if i > 547]
place
for i in place_reviews[549]:
    conn.execute(f"""INSERT INTO reviews (place_id, author_name, author_url, original_language, translated, utc_time, rating, review_text)
         VALUES ('{reviews["place_id"][i]}', '{reviews["author_name"][i]}', '{reviews["author_url"][i]}', '{reviews["original_language"][i]}',
                 {reviews["translated"][i]}, {reviews["time"][i]}, {reviews["rating"][i]}, '{reviews["text"][i]}' );
        """)
    sleep(1)
    if i % 14==0:
        print(f"Index {i}: complete") ## Did not Forgot the f to make the substitution work this time.

Index 560: complete


In [473]:
conn=engine.connect()

In [477]:
photos_df=photos.loc[[i for i in photos.index if photos['place_id'][i] in good_places],]
photos_df

,photo_reference,place_id,height,width,uploader_name,uploader_url
10,AcYSjRipHf4GRJXIhpdCOC0Ni08bge0Q5RZJnyg6T3Ifs_...,ChIJm-BEGj4GwFQRQz0MEeIFSLM,1667,2500,Cória Estates,https://maps.google.com/maps/contrib/110200706...
11,AcYSjRjno6ghLAOTYXzgJtnIhzSfDULHR7qL0eGqZbPrr2...,ChIJm-BEGj4GwFQRQz0MEeIFSLM,1000,1500,Cória Estates,https://maps.google.com/maps/contrib/110200706...
12,AcYSjRhI443fNcu7-16x-m3p7CUn3iw5lYAcsxor0aeUwH...,ChIJm-BEGj4GwFQRQz0MEeIFSLM,1000,1500,Cória Estates,https://maps.google.com/maps/contrib/110200706...
13,AcYSjRj1kaEO8CqLxJIuK_JoWGi3_L1jMCWaibyuqMzXMc...,ChIJm-BEGj4GwFQRQz0MEeIFSLM,1406,2500,Cória Estates,https://maps.google.com/maps/contrib/110200706...
14,AcYSjRjZHu2BuNGkGlGghBclPDgZicPZNXhyfZYnuuvKUy...,ChIJm-BEGj4GwFQRQz0MEeIFSLM,1280,1920,Cória Estates,https://maps.google.com/maps/contrib/110200706...
...,...,...,...,...,...,...
4976,AcYSjRjqdfYQtNMbZNiX-H0EP9j6F-I5JSN-fVvk7IIMaP...,ChIJsfC-ZK9BlVQRdC0UjH76x58,3024,4032,Vincent HT,https://maps.google.com/maps/contrib/101869745...
4977,AcYSjRjiP83ZQK2B_rCItb_gT8MnWg9eLz96obbthoInHG...,ChIJsfC-ZK9BlVQRdC0UjH76x58,3024,4032,Cedric Nash,https://maps.google.com/maps/contrib/105740641...
4978,AcYSjRjkHmOeTViw-oxy9VuAomcyn4XZQ7Hq3nRs5qNKTj...,ChIJsfC-ZK9BlVQRdC0UjH76x58,4000,2250,Monty K Reed,https://maps.google.com/maps/contrib/105400613...
4979,AcYSjRg69d50NGWrwKOS05Gy-03idWwoYp6nJzbEFM3-Hc...,ChIJQaTQ6gxBlVQRGMITAAYqZ6U,3456,4608,Kevin Carr,https://maps.google.com/maps/contrib/106173123...
